In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchvision import datasets,transforms

from torch.utils.data import DataLoader,random_split

C:\Users\Aditya Gupta\Anaconda\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
transform = transforms.Compose([transforms.Resize((64,64)),transforms.ToTensor()])

In [3]:
dir='NN'
dataset=datasets.ImageFolder(dir,transform=transform)

In [4]:
tr_q=int(0.9*len(dataset))
te_q=len(dataset)-tr_q

In [5]:
train, test = random_split(dataset, [tr_q, te_q])
train_loader = DataLoader(train,batch_size=32, shuffle=True)
test_loader = DataLoader(test, batch_size=32, shuffle=False)

In [6]:
class BartHomer(nn.Module):
    def __init__(self):
        super(BartHomer,self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64*64*3, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 16)
        self.fc4 = nn.Linear(16,2)
    def forward(self,x):
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.softmax(self.fc4(x), dim=1)
        return x

In [7]:
model=BartHomer()

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [12]:
epochs=10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}')

Epoch 1/10, Loss: 0.6918
Epoch 2/10, Loss: 0.6827
Epoch 3/10, Loss: 0.6775
Epoch 4/10, Loss: 0.6715
Epoch 5/10, Loss: 0.6671
Epoch 6/10, Loss: 0.6610
Epoch 7/10, Loss: 0.6519
Epoch 8/10, Loss: 0.6401
Epoch 9/10, Loss: 0.6266
Epoch 10/10, Loss: 0.6101


In [13]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on test set: {accuracy:.2f}%')

Accuracy on test set: 92.59%
